In [101]:
# from http://code.activestate.com/recipes/578608-generates-tuples-of-integers-with-a-given-sum/
from itertools import permutations

def tuples_sum(nbval,total,order=True) :
    """ 
        Generate all the tuples L of nbval positive or nul integer 
        such that sum(L)=total.
        The tuples may be ordered (decreasing order) or not
    """
    if nbval == 0 and total == 0 : yield tuple() ; return #raise StopIteration
    if nbval == 1 : yield (total,) ; return #raise StopIteration
    if total==0 : yield (0,)*nbval ; return #raise StopIteration
    for start in range(total,0,-1) :
        for qu in tuples_sum(nbval-1,total-start) :
            if qu[0]<=start :
                sol=(start,)+qu
                if order : yield sol
                else :
                    l=set()
                    for p in permutations(sol,len(sol)) :
                        if p not in l :
                            l.add(p)
                            yield p
    
if __name__=='__main__' :
    print("How to obtain 5 by adding a+b+c (>=0) ? ")
    print("Give me the list of (a,b,c) tuples.")
    g=tuples_sum(3,6,order=False)
    print(list(g))
    
    print("How to obtain 6 by adding 3 positive or nul integers ?")
    g=tuples_sum(3,6,order=True)
    print(list(g))

How to obtain 5 by adding a+b+c (>=0) ? 
Give me the list of (a,b,c) tuples.
[(6, 0, 0), (0, 6, 0), (0, 0, 6), (5, 1, 0), (5, 0, 1), (1, 5, 0), (1, 0, 5), (0, 5, 1), (0, 1, 5), (4, 2, 0), (4, 0, 2), (2, 4, 0), (2, 0, 4), (0, 4, 2), (0, 2, 4), (4, 1, 1), (1, 4, 1), (1, 1, 4), (3, 3, 0), (3, 0, 3), (0, 3, 3), (3, 2, 1), (3, 1, 2), (2, 3, 1), (2, 1, 3), (1, 3, 2), (1, 2, 3), (2, 2, 2)]
How to obtain 6 by adding 3 positive or nul integers ?
[(6, 0, 0), (5, 1, 0), (4, 2, 0), (4, 1, 1), (3, 3, 0), (3, 2, 1), (2, 2, 2)]


In [102]:
n = 4
m = 7
list(tuples_sum(m,n,order=False))


[(4, 0, 0, 0, 0, 0, 0),
 (0, 4, 0, 0, 0, 0, 0),
 (0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 4, 0, 0, 0),
 (0, 0, 0, 0, 4, 0, 0),
 (0, 0, 0, 0, 0, 4, 0),
 (0, 0, 0, 0, 0, 0, 4),
 (3, 1, 0, 0, 0, 0, 0),
 (3, 0, 1, 0, 0, 0, 0),
 (3, 0, 0, 1, 0, 0, 0),
 (3, 0, 0, 0, 1, 0, 0),
 (3, 0, 0, 0, 0, 1, 0),
 (3, 0, 0, 0, 0, 0, 1),
 (1, 3, 0, 0, 0, 0, 0),
 (1, 0, 3, 0, 0, 0, 0),
 (1, 0, 0, 3, 0, 0, 0),
 (1, 0, 0, 0, 3, 0, 0),
 (1, 0, 0, 0, 0, 3, 0),
 (1, 0, 0, 0, 0, 0, 3),
 (0, 3, 1, 0, 0, 0, 0),
 (0, 3, 0, 1, 0, 0, 0),
 (0, 3, 0, 0, 1, 0, 0),
 (0, 3, 0, 0, 0, 1, 0),
 (0, 3, 0, 0, 0, 0, 1),
 (0, 1, 3, 0, 0, 0, 0),
 (0, 1, 0, 3, 0, 0, 0),
 (0, 1, 0, 0, 3, 0, 0),
 (0, 1, 0, 0, 0, 3, 0),
 (0, 1, 0, 0, 0, 0, 3),
 (0, 0, 3, 1, 0, 0, 0),
 (0, 0, 3, 0, 1, 0, 0),
 (0, 0, 3, 0, 0, 1, 0),
 (0, 0, 3, 0, 0, 0, 1),
 (0, 0, 1, 3, 0, 0, 0),
 (0, 0, 1, 0, 3, 0, 0),
 (0, 0, 1, 0, 0, 3, 0),
 (0, 0, 1, 0, 0, 0, 3),
 (0, 0, 0, 3, 1, 0, 0),
 (0, 0, 0, 3, 0, 1, 0),
 (0, 0, 0, 3, 0, 0, 1),
 (0, 0, 0, 1, 3, 0, 0),
 (0, 0, 0, 1, 0,

In [109]:
m = 2
n = 5
list(tuples_sum(m,n,order=False))
   

[(5, 0), (0, 5), (4, 1), (1, 4), (3, 2), (2, 3)]

In [222]:
'''
def generate_pair_partitions(inner_partition): #does not account for identical elements in inner_partition
    # base case
    if len(inner_partition) == 1:
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
        print(pair_partitions)
        pair_partitions_list = [[item] for item in pair_partitions]
        return pair_partitions_list
    pair_partitions_list = []
    pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
    print(pair_partitions)
    pair_partitions_list_old = generate_pair_partitions(inner_partition[1:])
    for i in range(len(pair_partitions)):
        extension = pair_partitions[i]
        for j in range(len(pair_partitions_list_old)):
            pair_partition_old = pair_partitions_list_old[j]
            pair_partition_new = [extension] + pair_partition_old 
            pair_partitions_list.append(pair_partition_new)
    return pair_partitions_list
'''

In [223]:
inner_partition = [2, 2, 2]
generate_pair_partitions(inner_partition)

[(2, 0), (0, 2), (1, 1)]
[(2, 0), (0, 2), (1, 1)]
[(2, 0), (0, 2), (1, 1)]


[[(2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (1, 1)],
 [(2, 0), (0, 2), (2, 0)],
 [(2, 0), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (1, 1)],
 [(2, 0), (1, 1), (2, 0)],
 [(2, 0), (1, 1), (0, 2)],
 [(2, 0), (1, 1), (1, 1)],
 [(0, 2), (2, 0), (2, 0)],
 [(0, 2), (2, 0), (0, 2)],
 [(0, 2), (2, 0), (1, 1)],
 [(0, 2), (0, 2), (2, 0)],
 [(0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (1, 1)],
 [(0, 2), (1, 1), (2, 0)],
 [(0, 2), (1, 1), (0, 2)],
 [(0, 2), (1, 1), (1, 1)],
 [(1, 1), (2, 0), (2, 0)],
 [(1, 1), (2, 0), (0, 2)],
 [(1, 1), (2, 0), (1, 1)],
 [(1, 1), (0, 2), (2, 0)],
 [(1, 1), (0, 2), (0, 2)],
 [(1, 1), (0, 2), (1, 1)],
 [(1, 1), (1, 1), (2, 0)],
 [(1, 1), (1, 1), (0, 2)],
 [(1, 1), (1, 1), (1, 1)]]

In [250]:
import inspect

In [261]:

def generate_pair_partitions(inner_partition, i_start):
    # base case
    print('')
    print('')
    print('NEW FUNCTION CALL')
    print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
    print('inner_partition, i_start: ' + str(inner_partition) + str(', ') + str(i_start))
    if len(inner_partition) == 1:
        print('len(inner_partition) == 1')
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list = [[item] for item in pair_partitions]
        print('RETURN')
        return pair_partitions_list
    
    pair_partitions_list = []
    pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
    print('pair_partitions: ' + str(pair_partitions))
    for i in range(i_start, len(pair_partitions)):
        print('i: ' + str(i))
        pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], i)
        print('pair_partitions_list_old: ' + str(pair_partitions_list_old))
        
        extension = pair_partitions[i]
        for j in range(len(pair_partitions_list_old)):
            pair_partition_old = pair_partitions_list_old[j]
            pair_partition_new = [extension] + pair_partition_old 
            pair_partitions_list.append(pair_partition_new)
    print('RETURN')
    return pair_partitions_list

In [263]:
inner_partition = [2, 2, 2, 2]
generate_pair_partitions(inner_partition, 0)



NEW FUNCTION CALL
RECURSION DEPTH: 0
inner_partition, i_start: [2, 2, 2, 2], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]
i: 0


NEW FUNCTION CALL
RECURSION DEPTH: 1
inner_partition, i_start: [2, 2, 2], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]
i: 0


NEW FUNCTION CALL
RECURSION DEPTH: 2
inner_partition, i_start: [2, 2], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]
i: 0


NEW FUNCTION CALL
RECURSION DEPTH: 3
inner_partition, i_start: [2], 0
len(inner_partition) == 1
RETURN
pair_partitions_list_old: [[(2, 0)], [(0, 2)], [(1, 1)]]
i: 1


NEW FUNCTION CALL
RECURSION DEPTH: 3
inner_partition, i_start: [2], 1
len(inner_partition) == 1
RETURN
pair_partitions_list_old: [[(0, 2)], [(1, 1)]]
i: 2


NEW FUNCTION CALL
RECURSION DEPTH: 3
inner_partition, i_start: [2], 2
len(inner_partition) == 1
RETURN
pair_partitions_list_old: [[(1, 1)]]
RETURN
pair_partitions_list_old: [[(2, 0), (2, 0)], [(2, 0), (0, 2)], [(2, 0), (1, 1)], [(0, 2), (0, 2)], [(0, 2), (1, 1)], [(1, 1), (1, 1)]]
i: 1


NEW FUNCTI

[[(2, 0), (2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (2, 0), (1, 1)],
 [(2, 0), (2, 0), (0, 2), (0, 2)],
 [(2, 0), (2, 0), (0, 2), (1, 1)],
 [(2, 0), (2, 0), (1, 1), (1, 1)],
 [(2, 0), (0, 2), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (0, 2), (1, 1)],
 [(2, 0), (0, 2), (1, 1), (1, 1)],
 [(2, 0), (1, 1), (1, 1), (1, 1)],
 [(0, 2), (0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (0, 2), (1, 1)],
 [(0, 2), (0, 2), (1, 1), (1, 1)],
 [(0, 2), (1, 1), (1, 1), (1, 1)],
 [(1, 1), (1, 1), (1, 1), (1, 1)]]

In [225]:


'''
def generate_pair_partitions(inner_partition, i_start):
    # INPUT:
    # inner_partition: sorted list (non-increasing order) of numbers that sum to some n_Di, an element of the outer
    # partition of n_D. 
    # OUTPUT:
    # pair_partitions_list: a list of lists of tuples, where each tuple has two elements that sum to the corresponding
    # element of the inner_partition. The number of tuples in each sublist matches the number of elements in 
    # inner_partition. 
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to partition elements of inner partition into two
    # non-negative integers (indicating number of derivatives acting on psi_bar and psi, respectively). 

    # base case
    if len(inner_partition) == 1:
        return [[pair_tuple] for pair_tuple in list(tuples_sum(2, inner_partition[0], order=False))[i_start:]] 

    print('')
    print('')
    # main body
    print('inner_partition, i_start: ' + str(inner_partition) + str(', ') + str(i_start))
    
    pair_partitions_list = [] # stores all distinct pair partitions 
    n = inner_partition[0] # number of derivatives in first element of inner partition
    pair_partitions = list(tuples_sum(2, n, order=False))[i_start:]
    
    print('pair_partitions: ' + str(pair_partitions))
    
    # if two successive elements of inner_partition are equal, then different orderings of 
    # corresponding pair partitions [..., (a,b), (c,d), ...] and [..., (c,d), (a,b), ...] are equal. 
    if inner_partition[1] == inner_partition[0]:
        
        print('')
        print('inner_partition[1] == inner_partition[0]')
        
        for i in range(len(pair_partitions)):
            
            #print('')
            print('i: ' + str(i))
            #print('pair_partitions[i]: ' + str(pair_partitions[i]))
            subpartitions_list = generate_pair_partitions(inner_partition[1:], i)
            print('subpartitions_list: ' + str(subpartitions_list))
            for j in range(len(subpartitions_list)):
                #print('j: ' + str(j))
                subpartition_extended = [pair_partitions[i]] + subpartitions_list[j] 
                #print(subpartition_extended)
                pair_partitions_list.append(subpartition_extended)
        
        return pair_partitions_list
    else:
        print('inner_partition[1] != inner_partition[0]')
        subpartitions_list = generate_pair_partitions(inner_partition[1:], i_start = 0)
        for i in range(len(pair_partitions)):
            for j in range(len(subpartitions_list)):
                subpartition_extended = [pair_partitions[i]] + subpartitions_list[j]
                #print(subpartition_extended)
                pair_partitions_list.append(subpartition_extended)
        return pair_partitions_list
'''

"\ndef generate_pair_partitions(inner_partition, i_start):\n    # INPUT:\n    # inner_partition: sorted list (non-increasing order) of numbers that sum to some n_Di, an element of the outer\n    # partition of n_D. \n    # OUTPUT:\n    # pair_partitions_list: a list of lists of tuples, where each tuple has two elements that sum to the corresponding\n    # element of the inner_partition. The number of tuples in each sublist matches the number of elements in \n    # inner_partition. \n    # EXPLANATION:\n    # recursive function that returns a list of all distinct ways to partition elements of inner partition into two\n    # non-negative integers (indicating number of derivatives acting on psi_bar and psi, respectively). \n\n    # base case\n    if len(inner_partition) == 1:\n        return [[pair_tuple] for pair_tuple in list(tuples_sum(2, inner_partition[0], order=False))[i_start:]] \n\n    print('')\n    print('')\n    # main body\n    print('inner_partition, i_start: ' + str(inner_pa

In [198]:
inner_partition = [2,2,2]
i_start = 0
generate_pair_partitions(inner_partition, i_start)



inner_partition, i_start: [2, 2, 2], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]

inner_partition[1] == inner_partition[0]
i: 0


inner_partition, i_start: [2, 2], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]

inner_partition[1] == inner_partition[0]
i: 0
subpartitions_list: [[(2, 0)], [(0, 2)], [(1, 1)]]
i: 1
subpartitions_list: [[(0, 2)], [(1, 1)]]
i: 2
subpartitions_list: [[(1, 1)]]
subpartitions_list: [[(2, 0), (2, 0)], [(2, 0), (0, 2)], [(2, 0), (1, 1)], [(0, 2), (0, 2)], [(0, 2), (1, 1)], [(1, 1), (1, 1)]]
i: 1


inner_partition, i_start: [2, 2], 1
pair_partitions: [(0, 2), (1, 1)]

inner_partition[1] == inner_partition[0]
i: 0
subpartitions_list: [[(2, 0)], [(0, 2)], [(1, 1)]]
i: 1
subpartitions_list: [[(0, 2)], [(1, 1)]]
subpartitions_list: [[(0, 2), (2, 0)], [(0, 2), (0, 2)], [(0, 2), (1, 1)], [(1, 1), (0, 2)], [(1, 1), (1, 1)]]
i: 2


inner_partition, i_start: [2, 2], 2
pair_partitions: [(1, 1)]

inner_partition[1] == inner_partition[0]
i: 0
subpartitions_list: [[(2, 0)], [(

[[(2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (1, 1)],
 [(2, 0), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (1, 1)],
 [(2, 0), (1, 1), (1, 1)],
 [(0, 2), (0, 2), (2, 0)],
 [(0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (1, 1)],
 [(0, 2), (1, 1), (0, 2)],
 [(0, 2), (1, 1), (1, 1)],
 [(1, 1), (1, 1), (2, 0)],
 [(1, 1), (1, 1), (0, 2)],
 [(1, 1), (1, 1), (1, 1)]]

In [135]:
inner_partition = [3,2,1]
i_start = 0
generate_pair_partitions(inner_partition, i_start)

inner_partition, i_start: [3, 2, 1], 0
pair_partitions: [(3, 0), (0, 3), (2, 1), (1, 2)]
inner_partition, i_start: [2, 1], 0
pair_partitions: [(2, 0), (0, 2), (1, 1)]


[[(3, 0), (2, 0), (1, 0)],
 [(3, 0), (2, 0), (0, 1)],
 [(3, 0), (0, 2), (1, 0)],
 [(3, 0), (0, 2), (0, 1)],
 [(3, 0), (1, 1), (1, 0)],
 [(3, 0), (1, 1), (0, 1)],
 [(0, 3), (2, 0), (1, 0)],
 [(0, 3), (2, 0), (0, 1)],
 [(0, 3), (0, 2), (1, 0)],
 [(0, 3), (0, 2), (0, 1)],
 [(0, 3), (1, 1), (1, 0)],
 [(0, 3), (1, 1), (0, 1)],
 [(2, 1), (2, 0), (1, 0)],
 [(2, 1), (2, 0), (0, 1)],
 [(2, 1), (0, 2), (1, 0)],
 [(2, 1), (0, 2), (0, 1)],
 [(2, 1), (1, 1), (1, 0)],
 [(2, 1), (1, 1), (0, 1)],
 [(1, 2), (2, 0), (1, 0)],
 [(1, 2), (2, 0), (0, 1)],
 [(1, 2), (0, 2), (1, 0)],
 [(1, 2), (0, 2), (0, 1)],
 [(1, 2), (1, 1), (1, 0)],
 [(1, 2), (1, 1), (0, 1)]]

In [ ]:
def generate_inner_partitions(outer_partition, field_multiplicities):
    # INPUT:
    # outer_partition: list of numbers that sum to number of derivatives n_D with n_fields elements. 
    # field_multiplicities: list of multiplicities of fields, same length as outer_partition
    # OUTPUT:
    # subpartitions_list: list of lists of tuples, with each tuple a sorted partition of the corresponding element
    # of outer partition
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to sub-partition elements of outer partition.
    # if there are k_1 distinct ways to partition n1, k_2 ways to partition n2, ..., k_d ways to partition nd,
    # then there are k_1 x k_2 x ... x k_d distinct ways to sub-partition elements of outer partition. 

    if len(outer_partition) != len(field_multiplicities):
        print('Error: Lengths of outer_partition and field_multiplicities arguments must be equal.')
        return None

    # base case
    if len(outer_partition)==1:
        n = outer_partition[0]
        m = field_multiplicities[0]
        return [[subpartition_tuple] for subpartition_tuple in tuples_sum(m, n, order=True)]

    # main body
    subpartitions_list = [] # stores all distinct subpartitions 
    n = outer_partition[0]
    m = field_multiplicities[0]
    inner_partitions = list(tuples_sum(m, n, order=True)) 
    subpartitions = generate_inner_partitions(outer_partition[1:], field_multiplicities[1:])
    for i in range(len(inner_partitions)):
        for j in range(len(subpartitions)):
            subpartition_extended = [inner_partitions[i]] + subpartitions[j] 
            subpartitions_list.append(subpartition_extended)

    return subpartitions_list  

In [113]:
def generate_derivative_assignments(field_multiset):
    # INPUT:
    # field_multiset: encodes the set of fields and derivatives making up the term
    #     e.g., field_multiset = {D:3, F:2, S:1, T:2}
    # OUTPUT:
    # derivative_assignments_list: A dictionary of lists of lists of tuples, where each sublist of each list is a 
    # distinct assignment of derivatives to fields.
    # EXPLANATION: generates all distinct ways of assigning derivatives to fields in field_multiset. The fields
    # S, V, T, Vp, Sp all contain two fields, a psi and a psi_bar. A derivative may be placed on either psi or psi_bar. 
    # More specifically, generate all 'outer partitions' among the field types - in the above example, F, psi_bar_S, psi_S, psi_bar_T, psi_T.
    # For each outer partition, generate all 'inner_partitions' or 'subpartitions.' Given the 
    # outer partition of n, (n1, n2, n3, n4, n5), where each of the five fields fi occurs with multiplicity mi, a 
    # subpartition is a list of tuples [(), (), (), (), ()], where the ith tuple is a sorted partition of ni with
    # mi elements. The function generates all such lists of tuples
    
    # extract number of derivatives
    n_D = field_multiset['D']
    
    
    # find length of outer partition 
    bilinear_keys = [k for k in list(field_multiset.keys()) if k != 'D' and k != 'F'] # deri
    bilinear_multiplicities = [field_multiset[m] for m in list(field_multiset.keys()) if m != 'D' and m != 'F']
    #n_fields = sum([1 if 'F' in field_multiset.keys() else 0]) \ # use list comprehension on list of one element 
    #    + sum([2 for key in bilinears]) # group psi and psi_bar together as one field for now
    
    if 'F' in field_multiset.keys():
        F_multiplicity = [field_multiset['F']]
    else: 
        F_multiplicity = []
    #bilinear_multiplicities = [x for pair in zip(bilinear_multiplicities,bilinear_multiplicities) for x in pair] 
    # for each bilinear, multiplicities are the same for psi and psi_bar
    field_multiplicities = [field_multiset[x] for x in field_multiset.keys() if x != 'D']
    
    n_fields = len(field_multiplicities)
    print('n_fields: ' + str(n_fields))
    print('bilinear_multiplicities: ' + str(bilinear_multiplicities))
    print('field_multiplicities: ' + str(field_multiplicities))
    
    
    #list of lists, each sublist an outer partition, where different orderings are distinct
    outer_partitions_list = list(tuples_sum(n_fields, n_D, order=False)) 
    
    # for each outer partition, generate a list of sorted inner partitions (ordering does not matter, so one may 
    # as well sort them) where the sum of each inner partition is the corresponding element of the outer partition.
    # this is a list of lists of lists. 
    
          
            
    derivative_assignments_dict = {} # for each outer partition key, a list of all ways of subpartitioning 
    # derivatives, where each way is a list of tuples, one tuple for each element of outer partition. 
    for outer_partition in outer_partitions_list:
        derivative_assignments_dict[outer_partition] = generate_inner_partitions(outer_partition, field_multiplicities)
        
    return derivative_assignments_dict

In [105]:
field_multiset = {'D':4, 'F':1, 'S':1, 'V': 2}
derivative_assignments = generate_derivative_assignments(field_multiset)
print('')
print('')
for i in range(len(derivative_assignments.values())):
    outer_partition  = list(derivative_assignments.keys())[i]
    print('outer partition: ' + str(outer_partition))
    subpartitions_list = derivative_assignments[outer_partition]
    for j in range(len(subpartitions_list)):
        subpartition = subpartitions_list[j]
        print('subpartition: ' + str(subpartition))
    print('')

n_fields: 5
bilinear_multiplicities: [1, 1, 2, 2]
field_multiplicities: [1, 1, 1, 2, 2]


outer partition: (4, 0, 0, 0, 0)
subpartition: [(4,), (0,), (0,), (0, 0), (0, 0)]

outer partition: (0, 4, 0, 0, 0)
subpartition: [(0,), (4,), (0,), (0, 0), (0, 0)]

outer partition: (0, 0, 4, 0, 0)
subpartition: [(0,), (0,), (4,), (0, 0), (0, 0)]

outer partition: (0, 0, 0, 4, 0)
subpartition: [(0,), (0,), (0,), (4, 0), (0, 0)]
subpartition: [(0,), (0,), (0,), (3, 1), (0, 0)]
subpartition: [(0,), (0,), (0,), (2, 2), (0, 0)]

outer partition: (0, 0, 0, 0, 4)
subpartition: [(0,), (0,), (0,), (0, 0), (4, 0)]
subpartition: [(0,), (0,), (0,), (0, 0), (3, 1)]
subpartition: [(0,), (0,), (0,), (0, 0), (2, 2)]

outer partition: (3, 1, 0, 0, 0)
subpartition: [(3,), (1,), (0,), (0, 0), (0, 0)]

outer partition: (3, 0, 1, 0, 0)
subpartition: [(3,), (0,), (1,), (0, 0), (0, 0)]

outer partition: (3, 0, 0, 1, 0)
subpartition: [(3,), (0,), (0,), (1, 0), (0, 0)]

outer partition: (3, 0, 0, 0, 1)
subpartition: [(3

In [106]:
len(generate_derivative_assignments(field_multiset).keys())

n_fields: 5
bilinear_multiplicities: [1, 1, 2, 2]
field_multiplicities: [1, 1, 1, 2, 2]


70